Advanced Visualization 1: Advanced Facet Grid

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# --- Advanced Visualization 1: Advanced Facet Grid ---

# Load the daily abnormal return data
try:
    df_daily_ar = pd.read_csv("./outcome/event_study_daily_ar.csv")
except FileNotFoundError:
    print("Error: 'event_study_daily_ar.csv' not found in the 'outcome' folder.")
    exit()

# --- Data Preparation ---
# We need to separate the 'Set' (_train/_test) from the 'EventGroup' name
# For example, 'internal_good_train' -> 'internal_good' and 'train'

df_plot_data = df_daily_ar.copy()
df_plot_data['Set'] = df_plot_data['EventGroup'].apply(lambda x: x.split('_')[-1])
df_plot_data['BaseEventGroup'] = df_plot_data['EventGroup'].apply(lambda x: '_'.join(x.split('_')[:-1]))

# Focus on one model for clarity
model_to_plot = 'MM_SPY'
df_plot_data = df_plot_data[df_plot_data['Model'] == model_to_plot]

# Calculate the Average Cumulative Abnormal Return (ACAR)
df_aar = df_plot_data.groupby(['Asset', 'BaseEventGroup', 'Set', 'RelativeDay'])['AR'].mean().reset_index()
df_aar = df_aar.sort_values(by='RelativeDay')
df_aar['ACAR'] = df_aar.groupby(['Asset', 'BaseEventGroup', 'Set'])['AR'].cumsum()

# --- Plotting ---
# Create a FacetGrid: rows are Assets, columns are BaseEventGroups.
# The lines within each subplot are colored by the Set (train vs test).
g = sns.FacetGrid(
    data=df_aar,
    row='Asset',
    col='BaseEventGroup',
    hue='Set',
    height=4,
    aspect=1.2,
    margin_titles=True,
    palette={'train': 'royalblue', 'test': 'darkorange'}
)

# Map the line plot to the grid
g.map(sns.lineplot, 'RelativeDay', 'ACAR', linewidth=2.5).add_legend(title='Set')

# --- Chart Enhancement ---
g.fig.suptitle(f'ACAR Path Comparison: Train vs. Test\nModel: {model_to_plot}', y=1.03)
g.set_axis_labels("Days Relative to Event", "Average Cumulative AR")
g.set_titles(row_template="{row_name}", col_template="{col_name}")

# Add reference lines to each subplot
g.map(plt.axhline, y=0, ls=":", c=".5")
g.map(plt.axvline, x=0, ls="--", c="red")

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

Advanced Visualization 2: Significance Annotation on a Bar Chart (Corrected)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# --- Advanced Visualization 2: Significance Annotation on a Bar Chart (Corrected) ---

# Step 1: Load the aggregated results data from the previous analysis
try:
    df_mean = pd.read_csv("./outcome/event_study_mean_results.csv")
except FileNotFoundError:
    print("Error: 'event_study_mean_results.csv' not found in the 'outcome' folder. Please run the main analysis script first.")
    # In a .py script, you might want to uncomment the next line to stop execution
    # exit()

# --- Step 2: Data Preparation ---

# Proceed only if the DataFrame was loaded successfully and is not empty
if 'df_mean' in locals() and not df_mean.empty:
    # --- Adjustable Parameters ---
    asset_to_plot = 'Bitcoin'
    model_to_plot = 'MM_SPY'
    window_to_plot = '(-10,+10)'
    # ---------------------------

    # Filter the DataFrame based on the parameters above
    # Using .copy() is recommended to avoid Pandas' SettingWithCopyWarning
    df_plot_data = df_mean[
        (df_mean['Asset'] == asset_to_plot) &
        (df_mean['Model'] == model_to_plot) &
        (df_mean['Window'] == window_to_plot)
    ].copy()

    # Helper function to convert a p-value into significance stars
    def p_to_stars(p):
        if pd.isna(p): return ''  # Return an empty string if p-value is missing
        if p <= 0.01: return '***'
        if p <= 0.05: return '**'
        if p <= 0.1: return '*'
        return '' # Return an empty string if not significant

    # --- Step 3: Plotting ---
    plt.figure(figsize=(14, 8))
    ax = sns.barplot(
        data=df_plot_data,
        x='EventGroup',
        y='MeanCAR',
        palette='Spectral' # Using the 'Spectral' color palette
    )

    # --- Step 4: Annotate Bars with Significance Stars ---
    # Iterate through each bar (patch) created by the barplot
    for i, p in enumerate(ax.patches):
        height = p.get_height()
        x_center = p.get_x() + p.get_width() / 2.
        
        # Get the corresponding p-value from the filtered DataFrame using the index 'i'
        # IMPORTANT: This assumes the row order in df_plot_data matches the plot order of the bars.
        # This is generally safe for sns.barplot when no explicit order is set.
        if 'p-value' in df_plot_data.columns:
            p_value = df_plot_data.iloc[i]['p-value']
            stars = p_to_stars(p_value)
        else:
            stars = '' # If the 'p-value' column doesn't exist, don't show stars
            if i == 0: # Print warning only once
                print("Warning: 'p-value' column not found. Significance stars will not be displayed.")

        # Determine the vertical offset for the annotation based on the bar's height
        y_offset = 10 if height >= 0 else -25 # Increase offset for negative bars to prevent overlap
        
        # Use ax.annotate() to place the text (stars) precisely on the plot
        ax.annotate(stars,
                    xy=(x_center, height), 
                    xytext=(0, y_offset),
                    textcoords='offset points',
                    ha='center',
                    fontsize=14,
                    fontweight='bold',
                    color='black')

    # --- Step 5: Chart Enhancement ---
    plt.title(f'Mean CAR with Significance Annotation for {asset_to_plot}\nModel: {model_to_plot}, Window: {window_to_plot}', fontsize=16)
    plt.xlabel('Event Group', fontsize=12)
    plt.ylabel('Mean CAR', fontsize=12)
    plt.axhline(0, ls='--', color='black', linewidth=0.8) # Add a horizontal line at y=0
    plt.xticks(rotation=45, ha='right') # Rotate x-axis labels for better readability
    plt.tight_layout() # Adjust plot to ensure everything fits without overlapping
    plt.show() # Display the final plot
    
else:
    print("DataFrame 'df_mean' was not loaded or is empty. Skipping plot.")

Advanced Visualization 3: Interactive Dashboard with Plotly (Corrected)

In [ ]:
import pandas as pd
import plotly.express as px

# --- Advanced Visualization 3: Interactive Dashboard with Plotly (Corrected) ---

# Load the aggregated results data
try:
    df_mean = pd.read_csv("./outcome/event_study_mean_results.csv")
except FileNotFoundError:
    print("Error: 'event_study_mean_results.csv' not found in the 'outcome' folder.")
    # In a .py script, you might want to uncomment the next line to stop execution
    # exit()

# --- Plotting ---
# Ensure the dataframe is loaded before plotting
if 'df_mean' in locals() and not df_mean.empty:
    fig = px.scatter(
        df_mean,
        x='Window',
        y='MeanCAR',
        facet_col='Asset',
        color='EventGroup',
        symbol='Model',
        size='N',
        # --- FIXED: Corrected the column name from 'p' to 'p-value' ---
        hover_data=['p-value', '95%CI_low', '95%CI_high'],
        # -----------------------------------------------------------------
        title='Interactive Analysis of Mean Cumulative Abnormal Returns (CAR)',
        labels={
            "Window": "Event Window",
            "MeanCAR": "Mean CAR",
            "EventGroup": "Event Group",
            "N": "Number of Events",
            "p-value": "p-value" # Also good to add a label for the corrected column
        },
        category_orders={"Window": ["(-1,+1)", "(-5,+5)", "(-10,+10)"]}
    )

    # --- Chart Enhancement ---
    fig.update_layout(
        legend_title_text='Click to Filter',
        title_x=0.5
    )
    fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='LightGrey')
    fig.update_traces(marker=dict(sizemin=5)) # Set a minimum marker size for better visibility

    # --- Display and Save ---
    # Use a renderer that opens the chart in a new browser tab.
    # This avoids potential issues with rendering inside notebooks.
    try:
        fig.show(renderer='browser')
    except Exception as e:
        print(f"Could not display figure. Error: {e}")


    # The 'write_html' method is the most robust way to save your interactive chart,
    # as it creates a portable file that can be opened on any computer with a web browser.
    try:
        output_path = "./outcome/interactive_car_dashboard.html"
        fig.write_html(output_path)
        print(f"\nInteractive dashboard saved to '{output_path}'")
    except Exception as e:
        print(f"\nCould not save HTML file. Error: {e}")
else:
    print("DataFrame 'df_mean' was not loaded or is empty. Skipping plot.")